In [ ]:
#Code for the first exercise of the experiment

In [28]:
import string
import numpy as np
from numpy import log10
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer, PunktSentenceTokenizer

In [29]:
'''Creating inverted index for a document'''

d1 = ['Herbivores are typically plant eaters and not meat eaters . ']
d2 = ['Carnivores are typically meat eaters and not plant eaters']
d3 = ['Deers eat grass and leaves']

stop_words = ['are', 'and', 'not']

docs = [d1, d2, d3]

def invert_index(docs = docs):
    dct = {}; TF = {}; C = []
    for i in range(len(docs)):
        docs[i] = PunktSentenceTokenizer().tokenize(docs[i][0])    #It takes str as input and outputs list of strings
        for sent in docs[i]:
            words = TreebankWordTokenizer().tokenize(sent)
            
        '''Creating inverted index'''
        for word in words:
            if word.lower() in stop_words or word in string.punctuation:
                continue
            
            word = PorterStemmer().stem(word)
            if word not in C:
                C.append(word)
                dct[word] = [i+1]
                TF[word] = [i+1]
            else:
                if i+1 not in dct[word]:
                    dct[word].append(i+1)
                '''For creating TF-IDF matrix'''
                TF[word].append(i+1)
    inverted_index = sorted(dct.items())
    return inverted_index, TF, dct

inverted_index, _, _ = invert_index()
print(f'Inverted Index: \n{inverted_index}')

Inverted Index: 
[('carnivor', [2]), ('deer', [3]), ('eat', [3]), ('eater', [1, 2]), ('grass', [3]), ('herbivor', [1]), ('leav', [3]), ('meat', [1, 2]), ('plant', [1, 2]), ('typic', [1, 2])]


In [42]:
'''TF-IDF term document Matrix'''

def TF_IDF():
    inverted_index, TF, dct = invert_index()
    sorted_dict = {k: v for k, v in sorted(dct.items(), key=lambda item: item[0])}
    term_list=list(sorted_dict.keys())
    print('The term list is as follows  :')
    print(term_list)
    
    TF = sorted(TF.items())
    # print('TF', TF)

    mat = [[0] * len(TF) for i in range(len(docs))]
    df = []
    for i in range(len(docs)):            #Iterating over all the words
        for j in range(len(TF)):      #Iterating over all the documents
            '''Term Frequency'''
            mat[i][j] = TF[j][1].count(i+1)
            '''TF-IDF without smoothing'''
            mat[i][j] = mat[i][j] * log10(len(docs)/len(inverted_index[j][1]))
    return mat

mat = TF_IDF()

#Columns represent words and each row represents a documnent word order is same as Inverted index
print('TF-IDF Matrix:')
print('TF-IDF for document 1',mat[0])
print()
print('TF-IDF for document 2',mat[1])
print()
print('TF-IDF for document 3',mat[2])

The term list is as follows  :
['carnivor', 'deer', 'eat', 'eater', 'grass', 'herbivor', 'leav', 'meat', 'plant', 'typic']
TF-IDF Matrix:
TF-IDF for document 1 [0.0, 0.0, 0.0, 0.3521825181113625, 0.0, 0.47712125471966244, 0.0, 0.17609125905568124, 0.17609125905568124, 0.17609125905568124]

TF-IDF for document 2 [0.47712125471966244, 0.0, 0.0, 0.3521825181113625, 0.0, 0.0, 0.0, 0.17609125905568124, 0.17609125905568124, 0.17609125905568124]

TF-IDF for document 3 [0.0, 0.47712125471966244, 0.47712125471966244, 0.0, 0.47712125471966244, 0.0, 0.47712125471966244, 0.0, 0.0, 0.0]


In [33]:
'''Document retrieval'''
def retrieve(query):
    _, _, dct = invert_index()
    query = TreebankWordTokenizer().tokenize(query)
    lst = []
    retrived = set()
    print('Document retrived for the query:\n')
    for i in range(len(query)):
        query[i] = PorterStemmer().stem(query[i])
        if query[i] in dct.keys():
            dummy = set(dct[query[i]])
            lst.append(dummy)
            if i == 1: 
                retrived = lst[1].intersection(lst[0])
            elif i>=1: 
                retrieved.intersection(lst[i])
            else: 
                retrieved = lst[0]
    return retrieved


In [34]:
query = 'Plant eaters,'
retrived = retrieve(query)
print(list(retrived))

Document retrived for the query:

[1, 2]


In [37]:
def cosine_similarity(query):
    inverted_index, _, dct = invert_index()
    sorted_dict = {k: v for k, v in sorted(dct.items(), key=lambda item: item[0])}
    term_list=list(sorted_dict.keys())
    query = TreebankWordTokenizer().tokenize(query)
    lst = []
    retrived = set()
    print('Document cosine similarities for the query:\n')
    for i in range(len(query)):
        query[i] = PorterStemmer().stem(query[i])
    query_vec=np.zeros(len(term_list))
    for i in range(len(term_list)):
        query_vec[i]=query.count(term_list[i])*(len(mat)/len(inverted_index[i][1]))
    doc_sim={}
    vector2 = np.array(query_vec)
    magnitude2 = np.linalg.norm(vector2)
    for i in range(len(mat)):
        vector1=np.array(mat[i])
        magnitude1=np.linalg.norm(vector1)
        dot_product = np.dot(vector1, vector2)
        doc_sim[i+1]=dot_product / (magnitude1 * magnitude2)
    
    return doc_sim

In [38]:
query = 'Plant eaters,'
print(cosine_similarity(query))

Document cosine similarities for the query:

{1: 0.5601569175157881, 2: 0.5601569175157881, 3: 0.0}
